# Minimale Spannbäume

## Tools für Experimente und Darstellung

Wir verwenden hier wieder `networkx` zur Repräsentation und zur Darstellung von Graphen.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from networkx.drawing.layout import spring_layout
import networkx as nx

Für den Algorithmus von Kruskal werden wir eine Union-Find-Struktur benötigen. Daher fügen wir hier eine Implementierung (vom letzten Mal) ein.

In [ ]:
class UnionFind: # Ranked quick-union with path compression
    def __init__(self, no_nodes):
        self.parent = list(range(no_nodes))
        self.components = no_nodes
        self.rank = [0] * no_nodes  # [0, ..., 0]

    def find(self, v):
        if self.parent[v] == v:
            return v
        root = self.find(self.parent[v])
        self.parent[v] = root
        return root

    def union(self, v, w):
        id_v = self.find(v)
        id_w = self.find(w)
        if id_v == id_w:
            return
        if self.rank[id_w] < self.rank[id_v]:
            self.parent[id_w] = id_v
        else:
            self.parent[id_v] = id_w
            if self.rank[id_v] == self.rank[id_w]:
                self.rank[id_w] += 1
        self.components -= 1

    def connected(self, v, w):
        return self.find(v) == self.find(w)

    def count(self):
        return self.components

Wir erstellen zufällig einen Beispielgraphen. Wir verwenden im Notebook die normale API von networkx-Graphen. Mit der alternative Kantenrepräsentation von den Folien unterstützen die Algorithmen auch Multigraphen (mit parallelen Kanten). Auch networkx hat eine geeignete Klasse `MultiGraph`, allerdings ist das Zeichnen paralleler Kanten im Notebook etwas kompliziert.

In [ ]:
import random
from itertools import combinations

def create_random_weighted_graph(no_nodes, no_edges, 
                                 min_edge_weight, max_edge_weight):
    graph = nx.Graph()
    node_numbers = list(range(no_nodes))
    graph.add_nodes_from(node_numbers)
    all_pairs = list(combinations(node_numbers, 2))
    possible_weights = list(range(min_edge_weight, max_edge_weight + 1))
    # print(all_pairs)
    for n1, n2 in random.sample(all_pairs, no_edges):
        weight = random.choice(possible_weights)
        graph.add_edge(n1, n2, weight=weight)
    return graph
        
graph = create_random_weighted_graph(10, 18, 1, 30)


In [ ]:
pos = spring_layout(graph)
nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue')
edge_labels = dict([((u,v,),d['weight']) for u,v,d in graph.edges(data=True)])
_ = nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

## Algorithmus von Prim

Der Algorithmus von Prim baut den Spannbaum ausgehend von einem bestimmten Knoten aus auf (wir wählen hier fest Knoten 0). Dabei betrachtet er jeweils eine Kante mit minimalem Gewicht unter allen am aktuellen Baum anliegenden Kanten. Verbindet sie einen neuen Knoten mit dem Baum, wird sie aufgenommen, sonst wird sie verworfen.

In [ ]:
class LazyPrim:
    def __init__(self, graph):
        self.included_edges = []
        self.total_weight = 0

        # node-indexed list: True if node already in tree
        included_nodes = [False] * graph.number_of_nodes()
        candidates = []

        # include an arbitrary node (we use 0) in tree
        included_nodes[0] = True
        for node in graph.neighbors(0):
            weight = graph.get_edge_data(0, node)["weight"]
            heapq.heappush(candidates, (weight, 0, node))

        self.dump(candidates)

        while (candidates and
               len(self.included_edges) < graph.number_of_nodes() - 1):
            weight, v, w = heapq.heappop(candidates)

            if included_nodes[v] and included_nodes[w]:
                print("skipped edge", (v, w))
                continue
            if included_nodes[w]:
                v, w = w, v
            # v is in tree, w is not
            included_nodes[w] = True
            self.included_edges.append((v, w))
            print("added edge", (v, w), "with weight", weight)
            self.total_weight += weight
            for node in graph.neighbors(w):
                if not included_nodes[node]:
                    weight = graph.get_edge_data(w, node)["weight"]
                    heapq.heappush(candidates, (weight, w, node))
            self.dump(candidates)

                    
    def edges(self):
        for e in self.included_edges:
            yield e

    def weight(self):
        return self.total_weight
    
    def dump(self, candidates=None):
        nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue', edge_color="gray")
        edge_labels=dict([((u,v,),d['weight']) for u,v,d in graph.edges(data=True)])
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        nx.draw_networkx_edges(graph, pos, edgelist=self.included_edges, edge_color="r")
        if candidates:
            edges = [(u,v) for (w, u, v) in candidates]
            nx.draw_networkx_edges(graph, pos, edgelist=edges, edge_color="b")
        plt.show()

In [ ]:
mst = LazyPrim(graph)
print("Der minimale Spannbaum hat Gewicht", mst.weight())